In [1]:
import tkinter as tk
from tkinter import messagebox
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import joblib
import numpy as np
import pandas as pd
model = joblib.load('resume_screening.joblib')
vectorizer = joblib.load('resume_tfidf.joblib')


def cleanResume(text):
    cleantext=re.sub('http\S+\s',' ',text)
    cleantext=re.sub('@\S+',' ',cleantext)
    cleantext=re.sub('#\S+',' ',cleantext)
    cleantext=re.sub('RT|cc',' ',cleantext)
    cleantext=re.sub('[%s]'% re.escape("""!#"$&'()*+,-./:;<=>?@[\]^_`{|}~"""),' ',cleantext)
    cleantext=re.sub(r'[^\x00-\x7f]',' ',cleantext)
    cleantext=re.sub('\s+',' ',cleantext)
    
    return cleantext



def remove_stopwords(input_text):
    words = word_tokenize(input_text)

    stop_words = set(stopwords.words('english'))

    filtered_words = [word for word in words if word.lower() not in stop_words]

    filtered_text = ' '.join(filtered_words)

    return filtered_text



def create_mapping(nums, texts):
    if len(nums) != len(texts):
        raise ValueError("Input arrays must have the same length.")

    mapping = dict(zip(nums, texts))
    return mapping



def resume_screening(text):
    
    numerical_values = [6, 12,  0,  1, 24, 16, 22, 14,  5, 15,  4, 21,  2, 11, 18, 20,  8, 17, 19,  7, 13, 10,  9,  3, 23]
    text_values = ['Data Science', 'HR', 'Advocate', 'Arts', 'Web Designing','Mechanical Engineer', 'Sales', 'Health and fitness','Civil Engineer', 'Java Developer', 'Business Analyst','SAP Developer', 'Automation Testing', 'Electrical Engineering','Operations Manager', 'Python Developer', 'DevOps Engineer','Network Security Engineer', 'PMO', 'Database', 'Hadoop','ETL Developer', 'DotNet Developer', 'Blockchain', 'Testing']
    result_mapping = create_mapping(numerical_values, text_values)
    text=cleanResume(text)
    text=remove_stopwords(text)
    text=np.array([text])
    text_vector=vectorizer.transform(text)
    res=model.predict(text_vector)
    
    return result_mapping[res[0]]



def on_predict_button_click():
    # Get the text from the text box
    input_text = text_entry.get("1.0",'end-1c')

    # Call the resume screening function
    result = resume_screening(input_text)

    # Display the result below the predict button
    result_label.config(text=result)

# Create the main Tkinter window
root = tk.Tk()
root.title("Resume Screening App")

# Create a text box for input
text_entry = tk.Text(root, height=10, width=50)
text_entry.pack(pady=10)

# Create a predict button
predict_button = tk.Button(root, text="Predict", command=on_predict_button_click)
predict_button.pack(pady=10)

# Create a label to display the result
result_label = tk.Label(root, text="")
result_label.pack()

# Start the Tkinter event loop
root.mainloop()
